In [159]:
from sympy import *
from IPython.display import display

In [160]:
#init_printing()

Resulting light = light coming from pixel * transmittance + integral over all points inbetween of (light coming from that point * transmittance from that point to camera)

Cryteks "exponential height fog" is used to simply scale the constants $\sigma_{a,c}$ and $\sigma_{s,c}$ depending on the height:

In [161]:
c, sigma_ac, sigma_sc = symbols('c sigma_ac sigma_sc')

In [162]:
def sigma_a(y):
    return sigma_ac*exp(-c*y)
def sigma_s(y):
    return sigma_sc*exp(-c*y)

In [163]:
def sigma_t(y):
    return sigma_a(y) + sigma_s(y)

Transmittance from pixel to camera:

In [164]:
o, t, d, D = symbols('o t d D')

In [213]:
i = Integral(sigma_t(o+t*d), (t, 0, D))
i

Integral(sigma_ac*exp(-c*(d*t + o)) + sigma_sc*exp(-c*(d*t + o)), (t, 0, D))

In [214]:
i.doit()

Piecewise(((-sigma_ac - sigma_sc)*exp(-c*(D*d + o))/(c*d) - (-sigma_ac - sigma_sc)*exp(-c*o)/(c*d), Ne(c*d, 0)), (D*(sigma_ac + sigma_sc), True))

After some transformations it is obvious that this is equal to the formula used in the crytek paper

In [167]:
(sigma_ac+sigma_sc)*exp(-c*o)*(1-exp(-c*D*d))/(c*d)

(1 - exp(-D*c*d))*(sigma_ac + sigma_sc)*exp(-c*o)/(c*d)

Then, the light from each point between the pixel and the camera has to be accumulated

In [168]:
l_i, t_2 = symbols('l_i t_2')

In [172]:
#Helper function to calculate transmittance from the origin along a direction up until a given depth
def transmittance(o, d, D):
    return exp(-Integral(sigma_t(o+t_2*d), (t_2, 0, D)))

In [200]:
def L_s(y): #in scatterd light at height
    return sigma_s(y)*l_i #just assume constant inscattering with a constanst phase function for this basic fog type

In [201]:
i = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i

Integral(l_i*sigma_sc*exp(-c*(d*t + o))*exp(-Integral(sigma_ac*exp(-c*(d*t_2 + o)) + sigma_sc*exp(-c*(d*t_2 + o)), (t_2, 0, t))), (t, 0, D))

In [207]:
i = i.doit()
i

Piecewise((-l_i*sigma_sc*exp(-(-sigma_ac - sigma_sc)*exp(-c*(D*d + o))/(c*d) + (-sigma_ac - sigma_sc)*exp(-c*o)/(c*d))/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc), (Ne(c*d, 0) & Ne(sigma_ac + sigma_sc, 0)) | (Ne(c*d, 0) & Ne(sigma_ac + sigma_sc, 0) & Ne(c, -sigma_ac/d - sigma_sc/d))), (-l_i*sigma_sc*exp(-c*(D*d + o))/(c*d) + l_i*sigma_sc*exp(-c*o)/(c*d), Ne(c*d, 0) | (Ne(c*d, 0) & Ne(c, -sigma_ac/d - sigma_sc/d))), (-l_i*sigma_sc*exp(-c*o)/(c*d*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*c*d) + sigma_ac*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*c*d) + sigma_sc*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*c*d)) + l_i*sigma_sc*exp(-c*o)/(c*d + sigma_ac + sigma_sc), Ne(c, -sigma_ac/d - sigma_sc/d) | (Ne(sigma_ac + sigma_sc, 0) & Ne(c, -sigma_ac/d - sigma_sc/d))), (l_i*sigma_sc*(D*sigma_ac/(sigma_ac + sigma_sc) + D*sigma_sc/(sigma_ac + sigma_sc))*exp(-c*o), True))

In [210]:
i2 = i.args[0][0]

In [211]:
simplify(i2)

l_i*sigma_sc*(1 - exp((sigma_ac + sigma_sc)*(exp(c*o) - exp(c*(D*d + o)))*exp(-c*(D*d + 2*o))/(c*d)))/(sigma_ac + sigma_sc)